# Synthetic Ensembles

This notebook shows how we created our synthetic ensembles. We use a mixture of real age ensembles and artificial, AR1 process noise.

In [1]:
import pickle

import pyleoclim as pyleo
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

from matplotlib.ticker import FormatStrFormatter

In [2]:
with open('../data/holo_chrons_study.pkl','rb') as handle:
    holo_chrons_study = pickle.load(handle)

with open('../data/pos_dict.pkl','rb') as handle:
    pos_dict = pickle.load(handle)

In [3]:
# define the function of a spike
def spike(t,delta,A):
    ''' Function to create a spike
        t : float
            time
        A : float
            amplitude
        delta : float
            parameter for the curve'''
    f=1/(2*(len(t)-1))
    y = (A/np.arctan(1/delta))*np.arctan(np.sin(2*np.pi*t*f)/delta)
    return y

def add_spike(series,xstart,xend,A,method='smooth'):
    '''Function to add a spike to a pyleoclim series
    
    Parameters
    ----------

        series : pyleo.Series; 
            the series to which the spike will be added
        xstart : float
            the starting year of the spike
        xend : float
            the ending year of the spike
        A : float
            the amplitude of the spike
        method : str
            "smooth": add a spike by spike;
            otherwise, directly add values of A at each timestep'''
    x = series.time
    y=series.value
    if method=='smooth':
        y[(x>=xstart)&(x<=xend)] += spike(np.arange(0,sum((x>=xstart)&(x<=xend))),0.02,A)
    else:
        y[(x>=xstart)&(x<=xend)] += [(x>=xstart)&(x<=xend)]+np.full(xend-xstart+1,A)

    series.value = y
    return series

In [4]:
series_dict = {}
ens_dict = {}
i=0
for label,record in holo_chrons_study.items():
    #Load series
    time = record['age']
    value = record['d18O']
    lat = pos_dict[label]['lat']
    lon = pos_dict[label]['lon']
    series = pyleo.GeoSeries(
        time = time,
        value=value,
        time_name = 'Age',
        time_unit = 'yrs BP',
        value_name = r'$\delta^{18} O$',
        value_unit = u'‰',
        label=label,
        lat = lat,
        lon=lon,
        archiveType='speleothem'
    ).interp(time_axis=time).detrend()

    series_dict[label] = series

    #Slice the series to the Holocene
    sliced = series.slice((0,10000))

    #Fit AR1 model
    g = pyleo.utils.ar1_fit(y=sliced.value,t=sliced.time)

    #Generate surrogate values according to ar1 model
    surr_value = pyleo.utils.tsmodel.ar1_model(t=series.time,tau=g)

    #Load ensemble
    chron = record['chron']
    ens_list = []

    for time_axis in chron:
        # Create surrogate series
        ens_series = pyleo.GeoSeries(
            time = time_axis,
            value = surr_value,
            time_name = 'Age',
            time_unit = 'yrs BP',
            value_name = r'$\delta^{18} O$',
            value_unit = u'‰',
            label=label,
            lat = lat,
            lon=lon,
            archiveType='speleothem'
        )

        # Add a spike to the series
        try:
            ens_series = add_spike(ens_series,4000,4200,np.std(ens_series.value)*2) #S/N ratio of 2
        # track the number of series that failed
        except:
            i +=1
            continue

        ens_list.append(ens_series)

    ens = pyleo.EnsembleSeries(ens_list)
    ens_dict[label] = ens


/var/folders/5k/0y4jsz592qq0y78c_0ddgcpm0000gn/T/ipykernel_55547/2083665507.py:10: UserWarning: auto_time_params is not specified. Currently default behavior sets this to True, which might modify your supplied time metadata.  Please set to False if you want a different behavior.
  series = pyleo.GeoSeries(
/Users/alexjames/miniconda3/envs/docs/lib/python3.10/site-packages/pyleoclim/core/geoseries.py:207: UserWarning: auto_time_params is not specified. Currently default behavior sets this to True, which might modify your supplied time metadata.  Please set to False if you want a different behavior.
  super().__init__(time, value, time_unit, time_name, value_name,
/var/folders/5k/0y4jsz592qq0y78c_0ddgcpm0000gn/T/ipykernel_55547/2083665507.py:40: UserWarning: auto_time_params is not specified. Currently default behavior sets this to True, which might modify your supplied time metadata.  Please set to False if you want a different behavior.
  ens_series = pyleo.GeoSeries(


Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis values sorted in ascending order
Time axis v

In [5]:
with open('../data/synthetic_ens_dict.pkl','wb') as handle:
    pickle.dump(ens_dict,handle)